# Answers to [Incubator Data Science Camp](https://www.thedataincubator.com/) admission challenge questions
Rong Huang <br>
2018

_________________________
<h1> <i> CHALLENGE 1:</i></h1> 

An ant is on a rectangular n×m grid with southwest-most point (0,0) and northeast-most point (m,n). Starting at (0,0), each time, the ant travels along a path walking north or east by a unit length on the grid with equal probability until it reaches (m,n). Define the deviation D of a path (from the straight path) as $max(x_m$−$y_n$,$y_n$−$x_m)$ for all points (x,y) along the path.
_____________________________________
<br>
#### Q: What is the mean of D when m=11 and n=7? ####

#### A: 0.5188941460

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import *
import pandas as pd

In [2]:
def distance(m, n):
    # returns the distance mapping on grids
    dist = np.zeros((n+1, m+1))
    for y in range(n+1):
        for x in range(m+1):
            dist[y, x] = abs(Decimal(x)/Decimal(m) - Decimal(y)/Decimal(n))
    return dist

def next_p(point, m, n):
    # this function returns next grid location during ant path search
    x = point[1]
    y = point[0]
    next_pos = []
    Npt = 0
    if x < m:
        next_pos.append((y, x+1))
        Npt += 1
    if y < n:
        next_pos.append((y+1, x))
        Npt += 1
    return Npt, next_pos

def probToPoint(point0, m, n):
    # If location "point0" is the max_distance, calculate total probability of all 
    # paths from (0,0) to "point0"
    dist = distance(m, n)
    D0 = dist[point0]
    x0 = point0[1]
    y0 = point0[0]
    probs = np.zeros((n+1, m+1))
    probs[0,0] = 1.0
    for y in range(y0+1):
        for x in range(x0+1):
            prob = probs[y,x]
            if prob > 0:
                Npt, next_pos = next_p((y, x), m, n)
                prob_fraction = Decimal(prob)/Decimal(Npt)
                for next_point in next_pos:
                    if dist[next_point] < D0:
                        if (next_point[0] <= y0) & (next_point[1] <= x0):
                            probs[next_point] = Decimal(probs[next_point]) + Decimal(prob_fraction)
                    elif (next_point[0] == y0) & (next_point[1] == x0):
                        probs[next_point] = Decimal(probs[next_point]) + Decimal(prob_fraction)
                    elif dist[next_point] == D0:
                        # because distances in "dist" come in "pair"
                        probs[next_point] = Decimal(probs[next_point]) + Decimal(0.5)*Decimal(prob_fraction)
    return probs[y0, x0]


def probTo2D(m, n):
    # mapping of "probToPoint"
    probTo = np.zeros((n+1, m+1))
    for y in range(n+1):
        for x in range(m+1):
            probTo[y, x] = probToPoint((y, x), m, n)
    return probTo


def probToEnd(point0, m, n):
    # If "point0" is the Max_D, the probability of the path from (0,0) to end (n, m)
    probTo = probTo2D(m, n)
    dist = distance(m, n)
    D0 = dist[point0]
    x0 = point0[1]
    y0 = point0[0]
    prob_fini = np.zeros((n+1, m+1))
    prob_fini[y0, x0] = probTo[y0, x0]  # start with the probability from (0,0) to this point
    for y in range(y0, n+1):
        for x in range(x0, m+1):
            prob = prob_fini[y, x]
            if prob > 0:
                Npt, next_pos = next_p((y, x), m, n)
                if Npt > 0:
                    prob_fraction = Decimal(prob)/Decimal(Npt)
                    for next_point in next_pos:
                        if dist[next_point] < D0:
                            prob_fini[next_point] = Decimal(prob_fini[next_point]) + Decimal(prob_fraction)
                        elif dist[next_point] == D0:
                            # because distances come in "pair"
                            prob_fini[next_point] = Decimal(prob_fini[next_point]) + Decimal(0.5)*Decimal(prob_fraction)
    return prob_fini[n, m]


def probability(m, n):
    # mapping the probability, cumulative density f, for each grid as max_D point
    dist = distance(m, n)
    probFini = np.zeros((n+1, m+1))
    for y in range(n+1):
        for x in range(m+1):
            probFini[y, x] = probToEnd((y, x), m, n)
        
    prob_sum = sum(sum(probFini))  
    print("total probability should be exactly 1. Check ! ", prob_sum)

    mn = (m+1)*(n+1)
    ant = pd.DataFrame(index=range(mn), columns=["positions", "distances", 'probability'])
    for y in range(n+1):
        for x in range(m+1):
            row_n = y*(m+1) + x
            ant.iloc[row_n] = [(y, x), dist[y,x], probFini[y,x]]
        
    ant = ant.sort_values(by="distances")

    Ds = ant['distances'].unique()
    Ps = np.zeros(Ds.shape)
    Cdfs = np.zeros(Ds.shape)
    for i in range(len(Ds)):
        Ps[i] = ant[ant["distances"]==Ds[i]]["probability"].sum()
        if i == 0:
            Cdfs[i] = Ps[i]
        else:
            Cdfs[i] = Cdfs[i-1] + Ps[i]   
    return Ds, Ps, Cdfs


def meanDecimal(Ds, Ps):
    # mean calculated with Decimal for better accuracy, in case needed
    value = 0
    for i in range(len(Ds)):
        value = Decimal(value) + Decimal(Ds[i])*Decimal(Ps[i])
    return value


def stdDecimal(Ds, Ps):
    # std calculated with Decimal for better accuracy, in case needed
    meanD = meanDecimal(Ds, Ps)
    value = 0.0
    for i in range(len(Ds)):
        value = Decimal(value) + ((Decimal(Ds[i]) - Decimal(meanD))**Decimal(2)) * Decimal(Ps[i])
    value = Decimal(value)**Decimal(0.5)
    return value

def conditionP(Ps, Ds, Pt, Pt_cond):
    # return cumulative Pt, with conditional point Pt_cond
    P_cond = Ps[Ds > Pt_cond]
    P_t = Ps[Ds > Pt]
    Pcond = sum(P_t)/sum(P_cond)
    return Pcond

    
m = 11
n = 7
Ds, Ps, Cdfs = probability(m, n)
meanD = meanDecimal(Ds, Ps)
stdD = stdDecimal(Ds, Ps)
condP = conditionP(Ps, Ds, 0.6, 0.2)
print('mean D: ', meanD)
print('std error: ', stdD)
print('conditioned probability: ', condP)

total probability should be exactly 1. Check !  1.0
mean D:  0.5188941460151176892454657971
std error:  0.1817855196893432136857319483
conditioned probability:  0.328313996544


#### Q: What is the standard deviation of D when m=11 and n=7?

#### A: 0.1817855197

#### Q: What is the mean of D when m=23 and n=31?

#### A: 0.3532883806

In [3]:
m = 23
n = 31
Ds, Ps, Cdfs = probability(m, n)
meanD = meanDecimal(Ds, Ps)
stdD = stdDecimal(Ds, Ps)
condP = conditionP(Ps, Ds, 0.6, 0.2)
print("for m=23, n=31: ")
print('mean D: ', meanD)
print('std error: ', stdD)
print('conditioned probability: ', condP) 

total probability should be exactly 1. Check !  1.0
for m=23, n=31: 
mean D:  0.3532883806846021237832167383
std error:  0.1369409723043562710496520315
conditioned probability:  0.0631262303718


#### Q: What is the standard deviation of D when m=23 and n=31?

#### A: 0.1369409723

#### Q: What is the conditional probability that D is greater than 0.6 given that it is greater than 0.2 when m=11 and n=7?

#### A: 0.3283139965

#### Q: What is the conditional probability that D is greater than 0.6 given that it is greater than 0.2 when m=23 and n=31?

#### A: 0.0631262304
<br>


___________________________
<h1><i> CHALLENGE 2: </i></h1>
___________________________________
<br>
The Stanford Open Policing Datasets provide information on traffic stops in different states. Please download the csv files for Montana and Vermont. You may also find the README file helpful as you work through the challenge questions. It contains useful information about the dataset, including column descriptions. 
<br>


#### Q: What proportion of traffic stops in Montana involved male drivers? In other words, divide the number of traffic stops involving male drivers by the total number of stops.

#### A: Male ratio:  0.6750723334

In [5]:
mt = pd.read_csv("MT-clean.csv\MT_cleaned.csv", engine='python')

# MT gender ratio:
mt_g = mt["driver_gender"].value_counts()
mt_gender = mt_g.values
gender_ratio = mt_gender[0]/(mt_gender[0] + mt_gender[1])
print('Male ratio: ', gender_ratio)

Male ratio:  0.675072333421


#### Q: How many more times likely are you to be arrested in Montana during a traffic stop if you have out of state plates?

#### A: The ratio of likely-arrested between out-of-state and in Montana is: 1.209512935

In [6]:
# out of state arrested ratio
mt_state = mt[["out_of_state", "is_arrested"]].dropna()
mt_in = mt_state[mt_state["out_of_state"]==False]
mt_out = mt_state[mt_state["out_of_state"]==True]
in_stop = len(mt_in)
out_stop = len(mt_out)
out = mt_out["is_arrested"].value_counts()
mtin = mt_in["is_arrested"].value_counts()
in_arrest = mtin.loc[True]
out_arrest = out.loc[True]
arrest_ratio = out_arrest/out_stop/(in_arrest/in_stop)
print('arrested ratio: ', arrest_ratio)

arrested ratio:  1.20951293515


#### Q: Perform a (χ2) test to determine whether the proportions of arrests in these two populations are equal. What is the value of the test statistic? ####

#### A: $\chi^2$ statistic: 126.2517277

In [13]:
from scipy.stats import chisquare

In [8]:
# star to calculate chi-square:
total_stop = len(mt_state)
total_arrest = in_arrest + out_arrest
in_expect = in_stop / total_stop * total_arrest
out_expect = out_stop /total_stop * total_arrest
chi_square = (in_expect - in_arrest)**2 / in_expect + (out_arrest - out_expect)**2 /out_expect
print('chi_square: ', chi_square)

# varify chi_square with toolkit:
# from scipy.stats import chisquare
expected = np.array([in_expect, out_expect])
observed = np.array([in_arrest, out_arrest])
chisquare, pvalue = chisquare(observed, expected)
print('scipy chisquare, pvalue: ', chisquare, pvalue)

chi_square:  126.25172771
scipy chisquare, pvalue:  126.25172771 2.70852435306e-29


#### Q: What proportion of traffic stops in Montana resulted in speeding violations? In other words, find the number of violations that include "Speeding" in the violation description and divide that number by the total number of stops (or rows in the Montana dataset).

#### A: 0.6581580399

In [9]:
# speeding violation analysis
mt_speed = mt[['violation_raw', 'violation']].dropna()
mt_speed['violation_search'] = mt_speed['violation'].apply(lambda x: x.lower().find('speeding'))
violation_yes = len(mt_speed[mt_speed['violation_search'] > -1])
speed_ratio = violation_yes/len(mt_speed)
print('speed ratio: ', speed_ratio)

speed ratio:  0.6581580398644923


#### Q: How much more likely does a traffic stop in Montana result in a DUI than a traffic stop in Vermont? To compute the proportion of traffic stops that result in a DUI, divide the number of stops with "DUI" in the violation description by the total number of stops.

#### A: result-in-DUI ratio, 4.054943765

In [11]:
# DUI ratios: 
mt_violation = mt[["violation"]].dropna()
mt_violation['DUI'] = mt_violation['violation'].apply(lambda x: x.find('DUI'))
dui_yes = len(mt_violation[mt_violation['DUI'] > -1])

vt = pd.read_csv("VT-clean.csv\VT_cleaned.csv", engine='python')
# vt.info()
vt_violation = vt[['violation']]
vt_violation = vt_violation.dropna()
vt_violation['DUI'] = vt_violation['violation'].apply(lambda x: x.find('DUI'))
vt_dui_y = len(vt_violation[vt_violation['DUI'] > -1])

dui_ratio = dui_yes/len(mt_violation)/(vt_dui_y/len(vt_violation))
print('DUI ratio: ', dui_ratio)

DUI ratio:  4.054943765214862


#### Q: What is the extrapolated, average manufacture year of vehicles involved in traffic stops in Montana in 2020? To answer this question, calculate the average vehicle manufacture year for each year's traffic stops. Extrapolate using a linear regression.

#### A: 2008.843166

In [12]:
from sklearn.linear_model import LinearRegression

In [14]:
# vehicle years:
mt_years = mt[['stop_date', 'vehicle_year']].dropna()
mt_years['vehicle_year'] = pd.to_numeric(mt_years['vehicle_year'], errors='coerce')
mt_years['st_years'] = mt_years['stop_date'].apply(lambda x: int(x[0:4]))
mt_years = mt_years.dropna()
mt_years = mt_years.sort_values(by=['st_years'])
vcyears = mt_years['vehicle_year'].values
styears = mt_years['st_years'].values
indexYrs = mt_years['st_years'].value_counts(sort=False).index
meanYear = []
for index1 in indexYrs:
    meanYear.append(mt_years[mt_years['st_years']==index1]['vehicle_year'].mean())
meanYear = np.array(meanYear)
d = np.stack((indexYrs, meanYear), axis=1)

train = pd.DataFrame(d, columns=['indexYrs', 'meanYear'])
lr = LinearRegression()
lr.fit(train[['indexYrs']], train['meanYear'])
a1 = lr.coef_
a0 = lr.intercept_
meanYr2020 = a0 + a1*2020
print('mean year in 2020: ', meanYr2020)

mean year in 2020:  [ 2008.84316596]


#### Q: What is the p-value of this linear regression?

#### A: 5.609148254e-08

In [15]:
# p-value of the regression:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(indexYrs, meanYear)
# null hypothesis: slope is zero
print('slope, intercept, p-value: ', slope, intercept, p_value)

slope, intercept, p-value:  0.717416899955 559.661028049 5.6091482537e-08


#### Q: Combining both the Vermont and Montana datasets, find the hours when the most and least number of traffic stops occurred. What is the difference in the total number of stops that occurred in these two hours? Hours range from 00 to 23. Round stop times down to compute this difference.

#### A: hourly maximum difference:  95344

In [16]:
# start to calculate the difference between hours
time1 = mt['stop_time']
time1= time1.dropna()
time = time1.apply(lambda x: int(x[:2]))
time1_hrs = time.value_counts()

time2 = vt['stop_time'].dropna()
time = time2.apply(lambda x: int(x[:2]))
time2_hrs = time.value_counts()

for i in time1_hrs.index:
    time1_hrs.loc[i] = time1_hrs.loc[i] + time2_hrs.loc[i]
diff = max(time1_hrs.values) - min(time1_hrs.values)
print('hourly maximum difference: ', diff)

hourly maximum difference:  95344


#### Q: We can use the traffic stop locations to estimate the areas of the counties in Montana. Represent each county as an ellipse with semi-axes given by a single standard deviation of the longitude and latitude of stops within that county. What is the area, in square kilometers, of the largest county measured in this manner? Please ignore unrealistic latitude and longitude coordinates.

#### A: Area of largest county, in squere km, 10.88634534, Lincoln County

In [17]:
# county area estimation, earth radius = 6371 km:
mt_loc = mt[['county_name', 'lon', 'lat']]
mt_loc = mt_loc.dropna()

mt_loc = mt_loc[(mt_loc['lon'] > -116.3) & (mt_loc['lon'] < -104.0)]
mt_loc = mt_loc[(mt_loc['lat'] > 44.3) & (mt_loc['lat'] < 49)]

ct_names = mt_loc.county_name.unique()
area = {}
max_size = 0
for name in ct_names:
    mt_ct = mt_loc[mt_loc['county_name'] == name]
    errors = mt_ct.std()
    size = errors.loc['lon'] * errors.loc['lat'] * 6.371**2 * np.pi
    if size > max_size:
        max_size = size
        max_name = name
    area[name] = name
print('max size, county name: ', max_size, max_name)

max size, county name:  10.8863453382 Lincoln County
